In [12]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [13]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [15]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        
        #print(target)
        
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

#model.forward(Variable(torch.Tensor([[4.0, 2.3, 4.0, 2.2, 5.1], [4.0, 2.3, 4.0, 2.2, 5.1], [4.0, 2.3, 4.0, 2.2, 5.1], [4.0, 2.3, 4.0, 2.2, 5.1], [4.0, 2.3, 4.0, 2.2, 5.1]])))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312037
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.299944
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.306442
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.301933
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.288444
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.291373
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.278467
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.281361
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.270806
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.266456
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.259573
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.255764
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.242811
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.245337
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.236965
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.203211
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.170974
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.138108
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.087436
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.019840
Train Epoch: 1 [12800/60000 (

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\ProgramData\Miniconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.1877, Accuracy: 9463/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.147211
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.280938
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.151509
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.201173
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.251879
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.261519
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.085525
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.156739
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.236249
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.149932
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.238993
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.230920
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.124602
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.238104
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.075393
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.161289
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.195448
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.080489
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.138313
Train Epoch: 2 [12

Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.049784
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.088004
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.134868
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.127989
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.118146
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.073166
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.091085
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.134842
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.100003
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.092357
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.042682
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.037806
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.228207
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.128093
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.052414
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.071477
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.211944
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.111870
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.200850
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.148954


Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.023329
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.011911
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.095736
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.025541
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.161473
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.129319
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.056960
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.038844
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.109812
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.040411
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.073398
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.078211
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.262671
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.079104
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.031120
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.035640
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.142412
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.159644
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.031598
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.097485


Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.018773
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.044048
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.090352
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.056329
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.100629
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.082746
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.032494
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.057462
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.175397
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.102318
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.069142
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.045178
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.070329
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.066127
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.023278
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.065378
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.024437
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.051351
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.068027
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.070192


Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.031549
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.103212
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.053681
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.034241
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.089810
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.032311
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.015368
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.047301
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.028341

Test set: Average loss: 0.0575, Accuracy: 9805/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.049761
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.032958
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.034031
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.025948
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.061738
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.031115
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.038158
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.005637
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.036321
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.031629
Train Epoch: